In [1]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
library("factoextra")
library("maptree")

corrplot 0.92 loaded


Attaching package: ‘pls’


The following object is masked from ‘package:corrplot’:

    corrplot


The following object is masked from ‘package:stats’:

    loadings



Attaching package: ‘reshape2’


The following objects are masked from ‘package:data.table’:

    dcast, melt


Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa

Loading required package: cluster

Loading required package: rpart



In [14]:
guideModules = as.data.frame(read.csv("./../TextFiles/ME_GuideModules_leiden_6_Modules.csv",
                                 stringsAsFactors = F, strip.white = T, header = TRUE, row.names=1))
geneModules = as.data.frame(read.csv("./../TextFiles/ME_GeneModules_leiden_11_Modules.csv",
                                 stringsAsFactors = F, strip.white = T, header = TRUE, row.names=1))
geneModules$GeneName = rownames(geneModules)
coefsSgn = as.data.frame(read.csv("./../MixedEffectLMOutputs/ME_SignificantBetaCoefs.csv",
                                  header=TRUE, row.names=1))

In [15]:
rownames(coefsSgn) <- sapply(rownames(coefsSgn), function(x){k=strsplit(x,"_")
                                      return(k[[1]][2])})

In [16]:
dim(coefsSgn)
dim(coefsAll)

[1]  329 1041

[1] 1035 6685

In [42]:
allTestRes <- list()

for (guideGroup in sort(unique(guideModules$GuideGroup))){
    for (geneGroup in sort(unique(geneModules$GeneGroup))){
        
        blockBetas <- coefsSgn[guideModules[guideModules$GuideGroup == guideGroup, "GuideName"], 
                               geneModules[geneModules$GeneGroup == geneGroup, "GeneName"]]
        blockBetas <- melt(blockBetas)$value
        allSampledBetas <- sample(melt(coefsSgn)$value,100000)
        meanDif <- mean(blockBetas) - mean(allSampledBetas)
        
        k <- wilcox.test(blockBetas, allSampledBetas, alternative = c("greater"))
        allTestRes <- lappend(allTestRes, c(guideGroup, geneGroup, "greater", k$p.value, meanDif))
        
        k <- wilcox.test(blockBetas, allSampledBetas, alternative = c("less"))
        allTestRes <- lappend(allTestRes, c(guideGroup, geneGroup, "less", k$p.value, meanDif))
    }
}

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; using all as measure variables

No id variables; usi

In [43]:
allTestRes <- data.frame(do.call(rbind, allTestRes), stringsAsFactors = FALSE)
colnames(allTestRes) <- c("guideGroup", "geneGroup", "lessOrGreater", "P_value", "MeanDiff")
allTestRes$P_value <- as.numeric(allTestRes$P_value)
allTestRes$FDR <- p.adjust(allTestRes$P_value)
allTestRes <- allTestRes[allTestRes$FDR < 0.05,]
allTestRes$guideGroup = paste0("K",allTestRes$guideGroup)
allTestRes

,guideGroup,geneGroup,lessOrGreater,P_value,MeanDiff,FDR
,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,K0,G0,greater,1.067072e-64,0.0162177672527473,1.195121e-62
3,K0,G1,greater,8.763210e-20,0.0069134649867374,8.587945e-18
6,K0,G10,less,1.311277e-10,-0.0125386014032122,1.127698e-08
7,K0,G2,greater,3.985479e-142,0.0208492541820591,4.902140e-140
9,K0,G3,greater,2.450448e-22,0.00847804394201543,2.474952e-20
12,K0,G4,less,9.616495e-258,-0.0296032676891902,1.230911e-255
14,K0,G5,less,3.394637e-20,-0.00960974204001127,3.360691e-18
15,K0,G6,greater,4.981272e-39,0.0105076755068009,5.280148e-37
18,K0,G7,less,0.000000e+00,-0.048968569085101,0.000000e+00


In [45]:
write.csv(allTestRes,"./../TextFiles/GuideModuleGeneModuleEffectSignificance.csv" , row.names=FALSE)